<a href="https://colab.research.google.com/github/jrhumberto/cd/blob/main/parte2_combina%C3%A7%C3%B5es_completas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 - Estratégia de Recursão

Provavelmente a mais óbvia e adequada, ganha em todos os aspectos da combinação completa (com exceção da parelelização dos loops). A recursão permite um código mais elegante e controlado enquanto as funções que se encerram varrem da memória as variáveis temporárias.

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
#Função de recursão. Pega a primeira palavra e combina ou separa em relacao ao
#resto da frade, recursivamente.
def recursao_combinacao(frase):
  if(len(frase)==1):
    return frase

  palavra_inicial = frase[0]
  ultimas_palavras = frase[1:]

  combinacoes_das_ultimas_palavras = recursao_combinacao(ultimas_palavras)

  combinador = ' '
  separador = ','
  separados = []
  combinados = []

  for combinacao in combinacoes_das_ultimas_palavras:
    combinados.append(combinador.join([palavra_inicial,combinacao]))
    separados.append(separador.join([palavra_inicial,combinacao]))
  return combinados + separados

In [ ]:
text = input('Digite o texto: ')

token_text = word_tokenize(text)

print(recursao_combinacao(token_text))

['eu gosto de café', 'eu gosto de,café', 'eu gosto,de café', 'eu gosto,de,café', 'eu,gosto de café', 'eu,gosto de,café', 'eu,gosto,de café', 'eu,gosto,de,café']


# 2 - Estratégia Bonus: combinação completa (pauzinhos e bolinhas):

Também conhecida por combinação com repetição, as combinações completas permitem uma gama de aplicações, dentre elas determinar todas as divisões e combinações das palavras de uma frase em n-gramas não homogêneos. Ainda não havia visto essa utilziação das combinações completas, por isso achei interessante ressaltar que temos uma aplicação bem prática delas em NLP.

Vale resaltar que o método é fraco em eficiência em relação a memória, porém é fácilmente paralelizavel por meio de seus loops.

Como bonus extra extra, também é apresentado a paralelização através do pacote parsl (poderia ser através de Mpi, opencl ou cuda, mas o parsl é menos popular, portanto vamos tratar de popularizá-lo!)

In [ ]:
!pip install parsl

In [ ]:
import parsl
parsl.load()

In [ ]:
import string
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import re
import itertools 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import os
from parsl.app.app import python_app, bash_app
from parsl.configs.local_threads import config
import numpy as np

@python_app
def complete_combination(index,token_text):
  combinator = ' '
  separator = ','
  combination = ''
  id = 0
  for i in range(len(index)-1):
    step = index[i+1]-index[i]-1
    if(step>0):
      if len(combination)>0:
        combination = combination + separator + combinator.join(token_text[id:id+step])
      else:
        combination =  combinator.join(token_text[id:id+step])
      id = id + step
  if(id<len(token_text)):
    if len(combination)>0:
      combination = combination + separator + combinator.join(token_text[id:len(token_text)])
    else:
      combination = combinator.join(token_text[id:len(token_text)])
  if combination not in combinations:
    combinations.append(combination)
  return combinations

In [ ]:
#input de texto
text = input('Digite o texto: ')

#limpando e dividindo o texto em tokens
text = text.lower()
text =  re.sub(r'\w*\d\w*', ' ', text)
text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
token_text = word_tokenize(text)


#criando a estratégia de combinações completas (2n-1 pauzinhos e bolinhas)
arr = range(2*len(token_text)-1)
k = len(token_text) -1

#criando as combinações para as posições dos pauzinhos
indexes = list(itertools.combinations(arr, k))


#Verificando como a frase ficou dividida pelos pauzinhos e adicionando no vetor de combinações
combinations = []
parallel_array = []

#loop paralelo pelos indices dos pauzinhos
for index in indexes:
  parallel_array.append(complete_combination(index,token_text))

for conbination in parallel_array:
  if conbination.result() not in combinations:
    combinations.append(conbination.result())

#adicionando por ultimo os tokens todos separados
combinations.append(separator.join(token_text))

#vizualizando
print(combinations)

Digite o texto: cafe e bom
['cafe e bom', 'cafe,e bom', 'cafe e,bom', [...], 'cafe,e,bom']
